1202. 経路の逐次観測＆ミクロなNWにこだわりすぎていた．もう少し全体を見て経路全体の推定ができるのではないか．脳内NNで経路わかるんだし，，

In [1]:
import numpy as np 
import pandas as pd 
import os

In [9]:
####### reading data #######
# inputも時間間隔20secで実行

read_folder = '/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/20230130_0845_20sec_end162021_under10step'
file_list = os.listdir(read_folder)

df_quater = pd.read_csv('/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/20230130_0845_20sec.csv')

# df_linkも作り直し
df_link = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_link_post.csv')
L = len(df_link)

# df_nodeは作り直し
df_node = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_node.csv')

df_ble = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_nw_detail/ble_nw.csv')

In [10]:
# matrix = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_matrix_post.csv')


###### 各linkのoとdの座標を入れておく配列 ###### # 線分とビーコンの距離に関しての情報は向きがあってもなくても変わらない
oddslink_loc_array = [] 
for i in range(len(df_link)):
    linkid = df_link.loc[i, 'linkid']
    O = df_link.loc[i, 'o']
    D = df_link.loc[i, 'd']

    x_o = df_node[df_node['nodeid'] == O]['x'].iloc[0]
    y_o = df_node[df_node['nodeid'] == O]['y'].iloc[0]
    z_o = df_node[df_node['nodeid'] == O]['floor'].iloc[0] 
    o_loc = np.array([x_o, y_o, z_o])

    x_d = df_node[df_node['nodeid'] == D]['x'].iloc[0]
    y_d = df_node[df_node['nodeid'] == D]['y'].iloc[0]
    z_d = df_node[df_node['nodeid'] == D]['floor'].iloc[0] # これでseriesではなく値として抽出できるはず
    d_loc = np.array([x_d, y_d, z_d])

    loc_tuple = (o_loc, d_loc)

    oddslink_loc_array.append(loc_tuple) # oddslink_loc_array[linkid//2][0]がlinkidのonode, [1]がdnode，[2]が中点，，，，

###### ビーコンとリンク線分の中点間距離を返す関数 ######
def shortest_distance_to_segment(p1, p2, x):
    p1 = np.array(p1)
    p2 = np.array(p2)
    x = np.array(x)
    
    segment = p2 - p1
    v1 = x - p1
    v2 = x - p2
    mid_point = (p1 + p2) / 2

    # 垂直ベクトルを計算
    v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
    distance = np.linalg.norm(x - mid_point)
    
    return distance

###### d_arrayの用意 ###### # 各リンク(の中点）とbleビーコンとの距離配列
d_array = np.zeros((len(df_link), len(df_ble)))

for i in range(len(df_link)):
    p_o = oddslink_loc_array[i][0] # o座標 # i=1の時
    p_d = oddslink_loc_array[i][1] # d座標

    for j in range(len(df_ble)):
        x_ap = df_ble.loc[j, 'x']
        y_ap = df_ble.loc[j, 'y']
        z_ap = df_ble.loc[j, 'floor']
        p_ap = np.array([x_ap, y_ap, z_ap])

        d_array[i, j] = shortest_distance_to_segment(p_o, p_d, p_ap) # ijはlink i+1, ビーコンj+1の距離

###### 混雑度dens，maxRSSI，meanRSSI，検出回数を各ビーコン各timestepごとに入れた配列（ビーコン数23*タイムステップ数） ######
### 20秒にするのでここやり直す．
dens_jt = np.zeros((23, 45), dtype = int)
max_jt = np.zeros((23, 45), dtype = int)
mean_jt = np.zeros((23, 45), dtype = int)
#count_jt = np.zeros((23, 45), dtype = int)

grouped = df_quater.groupby(['ID', 'timestep']) # listにして渡す必要がある
df_list = [group.reset_index(drop=True) for name, group in grouped]

for group in df_list:
    # beacon_idとtimestepを得る
    beacon_id = int(group.loc[0, 'ID'])
    timestep = int(group.loc[0, 'timestep'])

    # ユニークMACアドレス数，最大&平均RSSI
    unique_mac_count = group['MAC'].nunique()
    max_rssi = group['RSSI'].max()
    mean_rssi = group['RSSI'].mean()
    # countはそのタイムステップでの全検出数つまりgroupの長さそのもの
    #count = len(group)

    # numpy配列の要素はシーケンスダ
    dens_jt[beacon_id-1, timestep-1] = unique_mac_count
    max_jt[beacon_id-1, timestep-1] = max_rssi
    mean_jt[beacon_id-1, timestep-1] = mean_rssi
    #count_jt[beacon_id-1, timestep-1] = count


In [27]:
max_beacon = 5
dist_list = d_array[:, max_beacon-1]
dist_dict = {index + 1: value for index, value in enumerate(dist_list)}
print(dist_dict)

{1: 72.58869710396233, 2: 48.53920745387494, 3: 26.86782615415897, 4: 21.895390622227787, 5: 41.298528309127065, 6: 64.85919659731974, 7: 106.08894117791584, 8: 79.94271928230826, 9: 54.064261786323314, 10: 29.375947516637606, 11: 5.673412839728854, 12: 19.00471407966895, 13: 42.30564044249836, 14: 43.362367209368195, 15: 48.98271085291239, 16: 59.14016730615645, 17: 50.15451836574763, 18: 75.68720944947975, 19: 69.13220780685872, 20: 57.65638605783781, 21: 50.21480508774543, 22: 37.8527319027329, 23: 33.17113921770031, 24: 41.866551529834204, 25: 32.54285524120679, 26: 55.80344753015112, 27: 45.83037349018671, 28: 24.17245459195402, 29: 34.996990192019425, 30: 61.95231123413106, 31: 51.835545839703265, 32: 43.28539945813068, 33: 56.083096571073966, 34: 89.1024976543905, 35: 81.51499356560689, 36: 44.94022716064513, 37: 52.1126418275881, 38: 20.602847497370306, 39: 28.50884659627587, 40: 49.189430493240856, 41: 40.084269664045536, 42: 35.28835986908247, 43: 44.140156731146746, 44: 56.9

In [19]:
print(read_folder)
print(len(file_list))

/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/20230130_0845_20sec_end162021_under10step
1256


In [26]:
# 各個人に対して，timestepごとでの最大スコアのビーコンを並べる，というのでもいいかもしれない．そいうデータだけ集めるのでもいいかも？
# これを教師データにするとか？今から機械学習とかyabaikana
# ファジーな観測と，そうでない意味のある観測とがあるのでこれをどう区別するか．階が変わったところで階段を2timestep費やしているとみる．
####### 観測方程式（ビーコンの評価） ######
def beacon_eval(rssi, max_rssi, freq): # 同時刻内で捕捉されたビーコンを評価．最大rssiと観測頻度による．20secも幅があるので
    return (max_rssi/rssi + freq) 

RES = []
for file_name in file_list:
    if file_name.endswith('.csv'):

        res = []
        file_path = os.path.join(read_folder, file_name)
        df = pd.read_csv(file_path)

        grouped = df.groupby('timestep')
        groups = [group for name, group in grouped]
        groups = [data.reset_index(drop = True) for data in groups]

        nts = len(groups)
        # print(nts)

        for t in range(nts):
            dfit = groups[t]
            ts = int(dfit.loc[0, 'timestep'])
            count_tot = len(dfit) # 個人i時点tのcountの総和はdfitの長さそのもの
                        
            groupby_j = dfit.groupby('ID')
            groupby_js = [group.reset_index(drop=True) for name, group in groupby_j]

            # 各ビーコンのscoreを記録するためのdict
            dict = {}

            for j in range(len(groupby_js)): 
                dfitj = groupby_js[j]
                count = len(dfitj) # そのビーコンでの検出回数
                jid = dfitj.loc[0, 'ID'] # このビーコンのid
                rssi = dfitj['RSSI'].max() # そのビーコンでの最大観測強度

                max_rssi = max_jt[jid-1, ts-1] # tsでのbeaconのmaxのrssi
                freq = count / count_tot

                # このビーコンのスコアを計算して辞書に入れる
                dict[jid] = beacon_eval(rssi, max_rssi, freq)

            # dictに全部入っているので比較
            max_beacon = max(dict, key = dict.get) # score maxのビーコン
            res.append(max_beacon)
        RES.append(res)

print(RES)

[[4, 23, 15, 15, 16], [1, 6, 6, 6, 6, 20], [2, 2, 2, 2, 15, 15, 15, 16, 16], [2, 2, 22, 22, 15, 15, 16], [4, 15, 15, 23, 16, 16], [7, 21, 6, 6, 6, 6, 19, 20], [22, 22, 15, 15, 16], [1, 2, 2, 2, 2, 15, 15, 16], [1, 23, 7, 19, 19, 20, 20], [2, 2, 2, 2, 2, 15, 15, 15, 16], [2, 22, 23, 23, 19, 20, 20], [23, 16, 19, 19, 19, 7, 21], [4, 5, 6, 6, 20, 20, 20], [3, 22, 5, 15, 15, 23, 16, 16], [3, 3, 3, 2, 2, 2, 15, 16, 16], [6, 6, 6, 20, 7, 21], [5, 6, 6, 20, 20, 20, 21], [1, 6, 20, 7, 7, 21], [6, 6, 6, 7, 7, 21, 20], [2, 2, 15, 15, 16], [2, 2, 15, 15, 15, 16, 16], [4, 5, 5, 6, 20, 7, 7, 21, 21], [5, 15, 19, 19, 20], [2, 2, 2, 15, 15, 16], [1, 6, 6, 19, 7, 21, 20, 20], [5, 23, 18, 19, 19, 20, 20], [1, 5, 6, 6, 6, 20, 20, 21], [5, 6, 6, 20, 20, 20], [5, 23, 19, 19, 20], [4, 5, 23, 23, 19, 19, 7, 21, 21], [1, 5, 6, 6, 20, 20, 20, 20, 21], [5, 6, 6, 6, 19, 7, 21], [4, 15, 23, 16, 16], [19, 6, 6, 6, 20, 20, 21, 20], [22, 22, 23, 19, 19, 20], [5, 16, 19, 19, 20, 20], [22, 22, 15, 15, 16], [20, 20, 2

In [21]:
from collections import defaultdict

count_dict = defaultdict(int)

for res in RES:
    key = tuple(res)
    count_dict[key] += 1

for pattern, count in count_dict.items():
    print(f'パターン: {pattern}，出現回数: {count}')

パターン: (4, 23, 15, 15, 16)，出現回数: 2
パターン: (1, 6, 6, 6, 6, 20)，出現回数: 1
パターン: (2, 2, 2, 2, 15, 15, 15, 16, 16)，出現回数: 1
パターン: (2, 2, 22, 22, 15, 15, 16)，出現回数: 1
パターン: (4, 15, 15, 23, 16, 16)，出現回数: 1
パターン: (7, 21, 6, 6, 6, 6, 19, 20)，出現回数: 1
パターン: (22, 22, 15, 15, 16)，出現回数: 15
パターン: (1, 2, 2, 2, 2, 15, 15, 16)，出現回数: 1
パターン: (1, 23, 7, 19, 19, 20, 20)，出現回数: 1
パターン: (2, 2, 2, 2, 2, 15, 15, 15, 16)，出現回数: 1
パターン: (2, 22, 23, 23, 19, 20, 20)，出現回数: 1
パターン: (23, 16, 19, 19, 19, 7, 21)，出現回数: 1
パターン: (4, 5, 6, 6, 20, 20, 20)，出現回数: 14
パターン: (3, 22, 5, 15, 15, 23, 16, 16)，出現回数: 1
パターン: (3, 3, 3, 2, 2, 2, 15, 16, 16)，出現回数: 1
パターン: (6, 6, 6, 20, 7, 21)，出現回数: 1
パターン: (5, 6, 6, 20, 20, 20, 21)，出現回数: 2
パターン: (1, 6, 20, 7, 7, 21)，出現回数: 2
パターン: (6, 6, 6, 7, 7, 21, 20)，出現回数: 1
パターン: (2, 2, 15, 15, 16)，出現回数: 34
パターン: (2, 2, 15, 15, 15, 16, 16)，出現回数: 4
パターン: (4, 5, 5, 6, 20, 7, 7, 21, 21)，出現回数: 1
パターン: (5, 15, 19, 19, 20)，出現回数: 1
パターン: (2, 2, 2, 15, 15, 16)，出現回数: 21
パターン: (1, 6, 6, 19, 7, 21, 20, 20)，出現回数: 1
パター

In [25]:
from sklearn.cluster import KMeans
import numpy as np

# 例としてリストAを作成
# 長さを揃える
data = [
    [1, 1, 2, 2, 3, 3],
    [2, 2, 3, 3, 3, 3],
    [4, 5, 6, 7, 8, 8],
    [1, 1, 2, 3, 3, 3],
    [4, 4, 4, 4, 5, 6],
    [1, 2, 3, 3, 4, 4],
    [4, 5, 7, 8, 8, 8],
    [1, 2, 2, 3, 3, 4],
    [1, 1, 2, 3, 4, 4]
]

# 時系列データをフラット化してnumpy配列に変換（ベクトル化）
data_np = np.array([np.array(sublist) for sublist in data])

# K-meansクラスタリングを適用
n_clusters = 3  # クラスタ数を設定
kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(data_np)

# クラスタリングされた結果を取得
cluster_labels = kmeans.labels_
cluster_centers = kmeans.cluster_centers_

# 各クラスタの代表的なパターンを出力
for cluster_id in range(n_clusters):
    cluster_indices = np.where(cluster_labels == cluster_id)[0]
    representative_pattern = cluster_centers[cluster_id]
    print(f"クラスタ {cluster_id + 1} の代表的なパターン: {representative_pattern}")
    print(f"このクラスタに含まれるパターンの数: {len(cluster_indices)}")
    print(f"このクラスタに含まれるパターンのインデックス: {cluster_indices}")


クラスタ 1 の代表的なパターン: [4.  5.  6.5 7.5 8.  8. ]
このクラスタに含まれるパターンの数: 2
このクラスタに含まれるパターンのインデックス: [2 6]
クラスタ 2 の代表的なパターン: [1.16666667 1.5        2.33333333 2.83333333 3.33333333 3.5       ]
このクラスタに含まれるパターンの数: 6
このクラスタに含まれるパターンのインデックス: [0 1 3 5 7 8]
クラスタ 3 の代表的なパターン: [4. 4. 4. 4. 5. 6.]
このクラスタに含まれるパターンの数: 1
このクラスタに含まれるパターンのインデックス: [4]
